In [1]:
import sys
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from pyproj import CRS
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

In [2]:
def tif2array(input_file, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy array.
    inputs:
        input_file (str) : the name of input GeoTiff file.
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)

    if dataset is None:
        return None

    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)

    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)

    #image = image[2:-2,2:-2,:]

    return image

In [3]:
# option 0 - load directly as np array if sorted by id
#samples_0 = np.genfromtxt('/media/philipp/DATA/color_corr_matrix/20_st_poelten_wien/sample_values_wien.csv',delimiter=',', skip_header=1)
#samples_1 = np.genfromtxt('/media/philipp/DATA/color_corr_matrix/20_st_poelten_wien/sample_values_st_poelten.csv',delimiter=',', skip_header=1)

In [4]:
# option 1 - load through gepandas as geodataframe and convert it into a np array
path_file_0 = '/media/philipp/DATA/color_corr_matrix/20_st_poelten_wien/sample_points_values_wien.shp'
path_file_1 = '/media/philipp/DATA/color_corr_matrix/20_st_poelten_wien/sample_points_values_st_poelten.shp'
# read data
sample_0 = geopandas.read_file(path_file_0)
sample_1 = geopandas.read_file(path_file_1)
# convert to numpy
sample_0 = sample_0.iloc[:,1:5].to_numpy()
sample_1 = sample_1.iloc[:,1:5].to_numpy()

In [6]:
sample_0 = sample_0 / 255
sample_1 = sample_1 / 255

In [8]:
# add column of ones
sample_0h = np.ones((sample_0.shape[0],sample_0.shape[1]+1))
sample_0h[:,:-1] = sample_0

sample_1h = np.ones((sample_1.shape[0],sample_1.shape[1]+1))
sample_1h[:,:-1] = sample_1

In [9]:
sample_0h

array([[0.19215686, 0.23137255, 0.25098039, 0.48235294, 1.        ],
       [0.30980392, 0.37254902, 0.36470588, 0.61176471, 1.        ],
       [0.25490196, 0.32156863, 0.30196078, 0.59215686, 1.        ],
       ...,
       [0.34117647, 0.42352941, 0.41568627, 0.74901961, 1.        ],
       [0.36862745, 0.43921569, 0.4       , 0.70588235, 1.        ],
       [0.76078431, 0.75294118, 0.75294118, 0.65098039, 1.        ]])

In [13]:
ColorCorrectionMatrix_20 = np.linalg.lstsq(sample_0h, sample_1h)[0]

<ipython-input-13-1b3d855d72f9>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ColorCorrectionMatrix_20 = np.linalg.lstsq(sample_0h, sample_1h)[0]


In [14]:
ColorCorrectionMatrix_20

array([[ 1.00091965e+00,  2.55002222e-01,  2.40596402e-01,
         1.42303800e-01,  4.19690065e-16],
       [-5.08850968e-01,  2.89393514e-01,  1.03774063e-01,
        -3.71997905e-01,  3.33066907e-16],
       [ 2.30748069e-01,  2.53596152e-01,  6.43858902e-01,
         2.67420191e-02, -8.88178420e-16],
       [-1.24675245e-01, -2.08988555e-01, -3.12388633e-01,
         6.44483958e-01,  1.11022302e-16],
       [ 2.53974343e-01,  2.43811733e-01,  1.52369989e-01,
         4.37197793e-01,  1.00000000e+00]])

In [15]:
np.save("/media/philipp/DATA/color_corr_matrix/20_st_poelten_wien/ColorCorrectionMatrix_20", ColorCorrectionMatrix_20)

In [16]:
path_file_0 = '/media/philipp/DATA/color_corr_matrix/04_mariazell_st_poelten/sample_points_values_st_poelten.shp'
path_file_1 = '/media/philipp/DATA/color_corr_matrix/04_mariazell_st_poelten/sample_points_values_mariazell.shp'
# read data
sample_04_0 = geopandas.read_file(path_file_0)
sample_04_1 = geopandas.read_file(path_file_1)
# convert to numpy
sample_04_0 = sample_04_0.iloc[:,1:5].to_numpy()
sample_04_1 = sample_04_1.iloc[:,1:5].to_numpy()

In [23]:
# add column of ones
sample_04_0h = np.ones((sample_04_0.shape[0],sample_04_0.shape[1]+1))
sample_04_0h[:,:-1] = sample_04_0 / 255

sample_04_1h = np.ones((sample_04_1.shape[0],sample_04_1.shape[1]+1))
sample_04_1h[:,:-1] = sample_04_1 / 255

In [31]:
sample_04_0h

array([[0.40784314, 0.41568627, 0.27843137, 0.82745098, 1.        ],
       [0.36470588, 0.38431373, 0.29411765, 0.78823529, 1.        ],
       [0.44313725, 0.45882353, 0.30588235, 0.85098039, 1.        ],
       ...,
       [0.41960784, 0.38431373, 0.31372549, 0.63529412, 1.        ],
       [0.45882353, 0.48627451, 0.35294118, 0.83137255, 1.        ],
       [0.49411765, 0.50588235, 0.36078431, 0.85098039, 1.        ]])

In [ ]:
# convert st_poelten image to colour of wien

In [24]:
sample_04_0hc = np.matmul(sample_04_0h, ColorCorrectionMatrix_20)

In [43]:
sample_04_0hc[:100]

array([[0.41175504, 0.36579089, 0.21441727, 0.8813257 , 1.        ],
       [0.39305087, 0.35788539, 0.22313323, 0.86200323, 1.        ],
       [0.4285319 , 0.3893187 , 0.23770968, 0.88619964, 1.        ],
       [0.41741039, 0.37760615, 0.231025  , 0.85895676, 1.        ],
       [0.41183497, 0.39983584, 0.24184048, 0.85489616, 1.        ],
       [0.33838611, 0.29153209, 0.16579091, 0.71699574, 1.        ],
       [0.41286467, 0.3823261 , 0.23237129, 0.85620694, 1.        ],
       [0.45909441, 0.40785677, 0.33294862, 0.61775608, 1.        ],
       [0.39353615, 0.3451538 , 0.20546635, 0.81302955, 1.        ],
       [0.45609583, 0.39193451, 0.2641895 , 0.8533618 , 1.        ],
       [0.39440996, 0.36689805, 0.24291444, 0.82554562, 1.        ],
       [0.42054009, 0.40491553, 0.26028968, 0.89500104, 1.        ],
       [0.38821334, 0.33534823, 0.17558189, 0.89915632, 1.        ],
       [0.57255784, 0.5804622 , 0.5276103 , 0.52367532, 1.        ],
       [0.57864296, 0.5868959 , 0.

In [42]:
sample_04_1h.shape

(680, 5)

In [66]:
ColorCorrectionMatrix_04 = np.linalg.lstsq(sample_04_0hc[350:], sample_04_1h[350:])[0]

<ipython-input-66-836e689b2fab>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ColorCorrectionMatrix_04 = np.linalg.lstsq(sample_04_0hc[350:], sample_04_1h[350:])[0]


In [28]:
np.save("/media/philipp/DATA/color_corr_matrix/04_mariazell_st_poelten/ColorCorrectionMatrix_04", ColorCorrectionMatrix_04)

In [45]:
ColorCorrectionMatrix_04

array([[ 2.24890897e+00,  1.61976575e+00,  1.30554796e+00,
        -2.56537514e-01,  3.49862040e-15],
       [ 1.54209008e+00,  2.27167820e+00, -6.47245982e-01,
         2.33793720e+00, -2.10942375e-15],
       [-1.48224031e+00, -1.79580297e+00,  6.24800761e-01,
        -1.30551756e+00,  2.08166817e-16],
       [ 5.60903582e-02,  9.30645295e-02,  2.74509317e-01,
         1.19262775e+00, -1.66533454e-16],
       [-9.33839248e-01, -8.46054305e-01, -3.82568942e-01,
        -1.01422310e+00,  1.00000000e+00]])

In [67]:
ColorCorrectionMatrix_04

array([[            nan,             nan,             nan,
                    nan, -2.18434629e-16],
       [            nan,             nan,             nan,
                    nan,  2.22044605e-15],
       [            nan,             nan,             nan,
                    nan, -9.43689571e-16],
       [            nan,             nan,             nan,
                    nan,  0.00000000e+00],
       [            nan,             nan,             nan,
                    nan,  1.00000000e+00]])

In [35]:
sample_04_0 = geopandas.read_file(path_file_0)
sample_04_1 = geopandas.read_file(path_file_1)

In [36]:
sample_04_0

,id,2017250__1,2017250__2,2017250__3,2017250__4,geometry
0,1,104.0,106.0,71.0,211.0,POINT (523290.869 458928.114)
1,2,93.0,98.0,75.0,201.0,POINT (523320.131 458882.328)
2,3,113.0,117.0,78.0,217.0,POINT (523297.310 458912.152)
3,4,105.0,108.0,73.0,203.0,POINT (523345.191 458794.957)
4,5,112.0,124.0,75.0,209.0,POINT (523388.879 458862.863)
...,...,...,...,...,...,...
675,676,88.0,86.0,60.0,188.0,POINT (568309.063 459052.013)
676,677,112.0,101.0,85.0,186.0,POINT (568309.063 459045.442)
677,678,107.0,98.0,80.0,162.0,POINT (567923.949 459460.651)
678,679,117.0,124.0,90.0,212.0,POINT (567789.532 459475.790)


In [37]:
sample_04_1

,id,2017360__1,2017360__2,2017360__3,2017360__4,geometry
0,1,80.0,102.0,77.0,140.0,POINT (523290.869 458928.114)
1,2,35.0,53.0,58.0,98.0,POINT (523320.131 458882.328)
2,3,84.0,108.0,84.0,154.0,POINT (523297.310 458912.152)
3,4,91.0,106.0,85.0,119.0,POINT (523345.191 458794.957)
4,5,55.0,69.0,64.0,129.0,POINT (523388.879 458862.863)
...,...,...,...,...,...,...
675,676,68.0,84.0,64.0,124.0,POINT (568309.063 459052.013)
676,677,83.0,94.0,73.0,113.0,POINT (568309.063 459045.442)
677,678,115.0,129.0,116.0,144.0,POINT (567923.949 459460.651)
678,679,45.0,53.0,52.0,84.0,POINT (567789.532 459475.790)
